# Enron Email Database: Actionable item Detection


The Enron email dataset contains approximately 500,000 emails generated by employees of the Enron Corporation. It was obtained by the Federal Energy Regulatory Commission during its investigation of Enron's collapse.

>Dataset source: https://www.kaggle.com/wcukierski/enron-email-dataset


# 1. Objective
- Create a heuristics-based linguistic model for detecting actionable items from the email.
- Use the rule-based model to classify sentences to actionable sentence and non-actionable sentence.

# 2. Explore the Data

In [ ]:
#import nltk
#nltk.download()
# select download all

In [6]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
# import relevant packages
import pandas as pd
import os
import gc
import re
import numpy as np
from tqdm import tqdm

In [17]:
# to control the number of dataframe characters displayed
pd.options.display.max_colwidth = 200

In [2]:
# Load data
raw_dataDF = pd.read_csv('emails/emails.csv')

In [18]:
# lets see the data
raw_dataDF

,file,message
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nConte..."
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\n..."
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: leah.arsdall@enron.com\nSubject: Re: test\nMime-Version: ..."
3,allen-p/_sent_mail/1000.,"Message-ID: <13505866.1075863688222.JavaMail.evans@thyme>\nDate: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: randall.gay@enron.com\nSubject: \nMime-Version: 1.0\nCont..."
4,allen-p/_sent_mail/1001.,"Message-ID: <30922949.1075863688243.JavaMail.evans@thyme>\nDate: Thu, 31 Aug 2000 05:07:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: greg.piper@enron.com\nSubject: Re: Hello\nMime-Version: 1..."
...,...,...
517396,zufferli-j/sent_items/95.,"Message-ID: <26807948.1075842029936.JavaMail.evans@thyme>\nDate: Wed, 28 Nov 2001 13:30:11 -0800 (PST)\nFrom: john.zufferli@enron.com\nTo: kori.loibl@enron.com\nSubject: Trade with John Lavorato\n..."
517397,zufferli-j/sent_items/96.,"Message-ID: <25835861.1075842029959.JavaMail.evans@thyme>\nDate: Wed, 28 Nov 2001 12:47:48 -0800 (PST)\nFrom: john.zufferli@enron.com\nTo: john.lavorato@enron.com\nSubject: Gas Hedges\nMime-Versio..."
517398,zufferli-j/sent_items/97.,"Message-ID: <28979867.1075842029988.JavaMail.evans@thyme>\nDate: Wed, 28 Nov 2001 07:20:00 -0800 (PST)\nFrom: john.zufferli@enron.com\nTo: dawn.doucet@enron.com\nSubject: RE: CONFIDENTIAL\nMime-Ve..."
517399,zufferli-j/sent_items/98.,"Message-ID: <22052556.1075842030013.JavaMail.evans@thyme>\nDate: Tue, 27 Nov 2001 11:52:45 -0800 (PST)\nFrom: john.zufferli@enron.com\nTo: jeanie.slone@enron.com\nSubject: Calgary Analyst/Associat..."


In [19]:
# lets see the data shape
raw_dataDF.shape

(517401, 2)

In [44]:
raw_dataDF.iloc[128]

file                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## 2.1. Data Overview
<br>
All of the data is in the file emails.csv:</br>
<pre>
<b>emails.csv</b> contains 2 columns: file and message.<br />
<b>Size of emails.csv</b> - 1.32GB<br />
<b>Number of rows in Train.csv</b> = 517401<br />
</pre>

## 2.2. Lets see some rows to understand the structure of the dataset.
<br>

After going through the data i can see that all datapoints doesnot have 'cc' and 'bcc' sections

### 2.2.1 The content of message column at row 120

In [3]:
pd.options.display.max_colwidth = 1500
raw_dataDF.iloc[120]

file                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               allen-p/_sent_mail/202.
message    Message-ID: <26838693.1075855689682.JavaMail.evans@thyme>\nDate: Fri, 4 Aug 2000 07:00:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: chris.gaskill@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Al

**The printed text is pasted below in a more readable format**
<br>
Message-ID: <26838693.1075855689682.JavaMail.evans@thyme>\n<br>
Date: Fri, 4 Aug 2000 07:00:00 -0700 (PDT)\n<br>
From: phillip.allen@enron.com\n<br>
To: chris.gaskill@enron.com\n<br>
Subject: \n<br>
Mime-Version: 1.0\n<br>
Content-Type: text/plain; charset=us-ascii\n<br>
Content-Transfer-Encoding: 7bit\n<br>
X-From: Phillip K Allen\n<br>
X-To: Chris Gaskill\n<br>
X-cc: \nX-bcc: \n<br>
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail\n<br>
X-Origin: Allen-P\n<br>
X-FileName: pallen.nsf\n<br>
\n<br>
can you build something to look at historical prices from where we saved \n<br>
curves each night.\n<br>
\n<br>
Here is an example that pulls socal only.\n<br>
Improvements could include a drop down menu to choose any curve and a choice \n<br>
of index,gd, or our curves.\n<br>
\n<br>
<br>


The message contains the following sections:
<br>
*Message-ID, Date, From, To, Subject, Mime-Version, Content-Type, Content-Transfer-Encoding, X-From, X-To, X-cc, X-bcc, X-Folder, X-Origin, X-FileName, 'body'*
<br>


### 2.2.2 The content of message column at row 128

In [48]:
pd.options.display.max_colwidth = 1500
raw_dataDF.iloc[128]

file                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

**The printed text is pasted below in a more readable format**
<br>
Message-ID: <9496454.1075855378828.JavaMail.evans@thyme>\n<br>
Date: Fri, 27 Apr 2001 13:02:00 -0700 (PDT)\n<br>
From: phillip.allen@enron.com\n<br>
To: tim.heizenrader@enron.com\n<br>
Subject: \n<br>
Cc: tim.belden@enron.com\n<br>
Mime-Version: 1.0\n<br>
Content-Type: text/plain; charset=us-ascii\n<br>
Content-Transfer-Encoding: 7bit\n<br>
Bcc: tim.belden@enron.com\n<br>
X-From: Phillip K Allen\n<br>
X-To: Tim Heizenrader <Tim Heizenrader/PDX/ECT@ECT>\n<br>
X-cc: Tim Belden <Tim Belden/HOU/ECT@ECT>\n<br>
X-bcc: \nX-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n<br>
\n<br>
Tim,\n<br>
\n<br>
Can you authorize access to the west power site for Keith Holtz.  He is our Southern California basis trader and is under a two year contract.\n<br>
\n<br>
On another note, is it my imagination or did the SARR website lower its forecast for McNary discharge during May.  It seems like the flows have been lowered into the 130 range and there are fewer days near 170.  Also the second half of April doesn't seem to have panned out as I expected.  The outflows stayed at 100-110 at McNary.  Can you email or call with some additional insight?\n<br>
\n<br>
Thank you,\n<br>
\n<br>
Phillip
\n<br>
<br>


The message contains the following sections:
<br>
*Message-ID, Date, From, To, Subject, Cc, Mime-Version, Content-Type, Content-Transfer-Encoding, Bcc, X-From, X-To, X-cc, X-bcc, X-Folder, X-Origin, X-FileName, 'body'*
<br>


# 3. Data preprocessing


## 3.1 Deduplication

Remove duplicates if any?
- no duplicates found

In [54]:
#Deduplication of entries
print(raw_dataDF.shape)
raw_dataDF.drop_duplicates(subset={"file","message"}, keep='first', inplace=True)

# total number of datapoints
data_pnts = len(raw_dataDF)
data_pnts

(517401, 2)


(517401, 2)

## 3.2 Extract the 'content' text


- Dataset's message  has the email body.

**I want to extract the body section.**
<br>
- What i have observed is, each section is separated by '\n', i can use '\n' as an Delimiter to separate different sections. but i only need the email body and need not separate all sections.
- the body of the email starts with '\n\n' i will use that as delimiter, and to avoid spliting the email body(content) if we have '\n\n' inside the body. i will split on only the 1st occurance of '\n\n'.


> lets see what we get after spliting before writing the function

In [4]:
def get_body(DF1):
    return DF1.split('\n\n',1)[1]

raw_dataDF['body'] = raw_dataDF['message'].map(get_body)

In [85]:
raw_dataDF['body'][128]

"Tim,\n\nCan you authorize access to the west power site for Keith Holtz.  He is our Southern California basis trader and is under a two year contract.\n\nOn another note, is it my imagination or did the SARR website lower its forecast for McNary discharge during May.  It seems like the flows have been lowered into the 130 range and there are fewer days near 170.  Also the second half of April doesn't seem to have panned out as I expected.  The outflows stayed at 100-110 at McNary.  Can you email or call with some additional insight?\n\nThank you,\n\nPhillip"

## 3.3 Break text into sentences

**The objective:** Use the rule-based model to classify *sentences* to actionable sentence and non-actionable sentence.
<br>
<br>
The objective clearly mentions that i have to classify the sentences, and not the email.
<br>
Anyway that can be used to classify the emails, with minimum code.
<br>
I will focus on the sentences and not the email.
<br>
i will keep all the sentences together into a dataframe.
        
        

In [5]:
# sentence tokenization
#https://mailman.uib.no/public/corpora/2007-October/005426.html
# https://www.nltk.org/_modules/nltk/tokenize/punkt.html
import nltk
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [6]:
#Let's see how does it work
dfdf = tokenizer.tokenize(raw_dataDF['body'][128])

dfdf[0]
# some cleaning is required that i will do in the next section.

'Tim,\n\nCan you authorize access to the west power site for Keith Holtz.'

In [7]:
print(dfdf[0])

Tim,

Can you authorize access to the west power site for Keith Holtz.


In [8]:
# lets write the function to get all the sentences
def text_to_sentence(df_body):
    sent_list = []
    for ele in tqdm(df_body):
        temp1 = tokenizer.tokenize(ele)
        for sent in temp1:
            sent = sent.replace("\n\n", " ")
            sent = sent.replace("\n", " ")
            sent_list.append(sent)
    return sent_list        

In [9]:
sentence_list = text_to_sentence(raw_dataDF['body'])

100%|████████████████████████████████████████████████████████████████████████| 517401/517401 [04:11<00:00, 2061.31it/s]


In [18]:
sentence_list[:20]

['Here is our forecast',
 'Traveling to have a business meeting takes the fun out of the trip.',
 'Especially if you have to prepare a presentation.',
 'I would suggest holding the business plan meetings here then take a trip without any formal business meetings.',
 'I would even try and get some honest opinions on whether a trip is even desired or necessary.',
 'As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.',
 'Too often the presenter speaks and the others are quiet just waiting for their turn.',
 'The meetings might be better if held in a round table discussion format.',
 'My suggestion for where to go is Austin.',
 "Play golf and rent a ski boat and jet ski's.",
 'Flying somewhere takes too much time.',
 'test successful.',
 'way to go!!',
 '!',
 'Randy,  Can you send me a schedule of the salary and level of everyone in the  scheduling group.',
 'Plus your thoug

>I have decided to not clean/remove anything from the data at this point as i am going to use the grammatical structure of the data to classify the sentences.
<br>


## 3.4 save the sentences to a file.

In [10]:
data = {'sentence':sentence_list}

sentences_df = pd.DataFrame(data)

In [12]:
sentences_df.to_csv("sentence_file.csv", index = False)